In [ ]:
%load_ext autoreload

In [ ]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd
from pathlib import Path
import os

In [ ]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import pydrake.symbolic as sym
import time
from pydrake.all import CspaceFreeLine
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import VerificationOption

from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

In [ ]:
import logging
drake_logger = logging.getLogger("drake")
# drake_logger.setLevel(logging.DEBUG)

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [ ]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
model_file = FindResourceOrThrow("drake/manipulation/models/iiwa_description/iiwa7/iiwa7_with_box_collision.sdf")
box_file = FindResourceOrThrow("drake/C_Iris_Examples/assets/shelves.sdf")


parser = Parser(plant)


models = []
models.append(Parser(plant, scene_graph).AddModelFromFile(model_file))
# models.append(Parser(plant, scene_graph).AddModelFromFile(box_file))

sp = 0.4
x_fac = 1.2
locs = [ [0,0,0], 
        [x_fac*sp, 1.4*sp,0.4], [x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,-1.4*sp,0.4], [-x_fac*sp,1.4*sp,0.4], 
        [0.0 ,0 , 0.95], [0.0 ,0 , -0.05]] 
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.get_body(plant.GetBodyIndices(model)[0]).body_frame(),
                     RigidTransform(locs[idx]))
    idx +=1


plant.Finalize()


        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros_like(plant.GetPositionLowerLimits())

#compute limits in t-space
limits_t = []
for q in [plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits()]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star))

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


In [ ]:
visualizer.meshcat2.Delete("/prm")

In [ ]:
# draw prm
import prm
from pydrake.all import Rgba

# limits = [np.array(t_low), np.array(q_high)]

visualizer.meshcat2.Delete("/prm")

def plot_prm(nodes, adjacency_list, width, color = Rgba(0.0, 0.0, 1, 1), prefix = ""):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = np.append(nodes[node_idx, :],0)
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = np.append(nodes[adjacency_list[node_idx][edge_idx], :],0)
            name = f"/{prefix}prm/rm/line{plt_idx}"
            visualizer.meshcat2.SetLine(name,  np.hstack([pos1[:,np.newaxis], pos2[:, np.newaxis]]),
                                 line_width = width, rgba = color)
            plt_idx +=1
            
plotting_fn_handle = partial(plot_prm, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1])

def collision_bad(pos, col_func_handle):
    return 1-col_func_handle(pos[0], pos[1])

prm_col_fn_handle = partial(collision, col_func_handle = visualizer.col_func_handle_rational)
prm_col_fn_handle_bad = partial(collision_bad, col_func_handle = visualizer.col_func_handle_rational)

PRM = prm.PRM( 
            limits_t,
            num_points = 20,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = None
            )

PRM_bad = prm.PRM( 
            limits_t,
            num_points = 20,
            col_func_handle = prm_col_fn_handle_bad,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = None
            )

# PRM.add_start_end(start, target)
PRM.plot()
tot_num_edges = len(PRM.adjacency_list)* len(PRM.adjacency_list[0])
# path, sp_length = PRM.find_shortest_path()

# mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
# mat.wireframeLinewidth = 2.0
# num_waypoints = len(path)
# for idx in range(num_waypoints-1):
#     vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
# traj= utils.PWLinTraj(path, 5.0)

## Set up the sliders so we can move the plant around manually

In [8]:
sliders = []
for i in range(plant.num_positions()):
    sliders.append(widgets.FloatSlider(min=plant.GetPositionLowerLimits()[i],
                                       max=plant.GetPositionUpperLimits()[i], 
                                       value=0, description=f'q{i}'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = np.zeros(plant.num_positions())
def handle_slider_change(change, idx):
    q[idx] = change['new']
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


# visualizer.jupyter_cell()

FloatSlider(value=0.0, description='q0', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q1', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q2', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q3', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q4', max=2.96706, min=-2.96706)

FloatSlider(value=0.0, description='q5', max=2.0944, min=-2.0944)

FloatSlider(value=0.0, description='q6', max=3.05433, min=-3.05433)

In [9]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()
print(f"Number of pairs {len(pairs)}")

# gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
# get_name_of_gid = lambda gid : inspector.GetName(gid)
# gids.sort(key=get_name_of_gid)
# right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
# left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

# right_sweeper_fused_col_geom = right_sweeper_gids[2:]
# right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
# left_sweeper_fused_col_geom = left_sweeper_gids[4:]
# left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
# scene_graph.collision_filter_manager()\
#             .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

# right_sweeper_end_gid = right_sweeper_gids[-1]
# left_sweeper_end_gid = left_sweeper_gids[-1]
# id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
#                        ]
# visualizer.collision_pairs_of_interest = id_pairs_of_interest


Number of pairs 21


In [ ]:
t0 = time.time()
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kAffine, q_star,
                                set(), VerificationOption()
                               )
t1 = time.time()
print(f"Time to construct line certifier = {t1-t0}s")

INFO:drake:Building rational 0/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 0.016 s. Has degree: 3
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 0/336
INFO:drake:Building rational 1/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 0.036 s. Has degree: 3
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 1/336
INFO:drake:Building rational 2/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 0.03 s. Has degree: 3
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 2/336
INFO:drake:Building rational 3/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO

INFO:drake:Done rational 27/336
INFO:drake:Building rational 28/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 0.08 s. Has degree: 3
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 28/336
INFO:drake:Building rational 29/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 0.062 s. Has degree: 3
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 29/336
INFO:drake:Building rational 30/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 0.069 s. Has degree: 3
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 30/336
INFO:drake:Building rational 31/336
INFO:drake:numerator to expression 0.0 s
INFO:dr

INFO:drake:numerator poly constructed in 311.484 s. Has degree: 7
INFO:drake:denominator evaluated in 0.017 s
INFO:drake:rational emplaced in -0.031 s
INFO:drake:Done rational 55/336
INFO:drake:Building rational 56/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 7.527 s. Has degree: 5
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 56/336
INFO:drake:Building rational 57/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 7.402 s. Has degree: 5
INFO:drake:denominator evaluated in 0.0 s
INFO:drake:rational emplaced in 0.0 s
INFO:drake:Done rational 57/336
INFO:drake:Building rational 58/336
INFO:drake:numerator to expression 0.0 s
INFO:drake:numerator substituted in 0.0 s
INFO:drake:numerator poly constructed in 7.374 s. Has degree: 5
INFO:drake:denominator evaluated in 0.0 s
IN

In [ ]:
s0 = PRM.nodes[PRM.adjacency_list[0][0]]
uncertified_edges = []
certified_edges = []
print(s0)
t0_tot = time.time()
num_safe = 0
ctr = 0
for item_num, item in enumerate(PRM.adjacency_list):
    s0 = PRM.nodes[item[0]]
    uncertified_edges.append([item[0]])
    certified_edges.append([item[0]])
    for idx1 in item[1:]:
        if (ctr % 100 == 0):
            print(f"Doing {ctr}/{tot_num_edges}")
        s1 = PRM.nodes[idx1]
        t0 = time.time()
        safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1)
        t1 = time.time()
        if not safe:
            uncertified_edges[-1].append(idx1)
        else:
            certified_edges[-1].append(idx1)
            num_safe += 1
        ctr += 1
    break;
t1_tot = time.time()
print(f"Certification in {t1_tot-t0_tot}s")
print(f"Certified {num_safe}/{ctr} edges")

In [ ]:
plot_prm(PRM.nodes, certified_edges, 0.2, color = Rgba(0.0, 1.0, 0.0, 1), prefix = "affine_certified_")
plot_prm(PRM.nodes, uncertified_edges, 0.2, color = Rgba(1.0, 0.0, 0.0, 1), prefix = "affine_uncertified_")

In [ ]:
line_certifier = CspaceFreeLine(diagram, plant, scene_graph,
                                rational_forward_kinematics.SeparatingPlaneOrder.kConstant, np.zeros(2),
                                set(), VerificationOption()
                               )
s0 = PRM.nodes[PRM.adjacency_list[0][0]]
uncertified_edges = []
certified_edges = []
print(s0)
t0_tot = time.time()
num_safe = 0
ctr = 0
for item_num, item in enumerate(PRM.adjacency_list):
    s0 = PRM.nodes[item[0]]
    uncertified_edges.append([item[0]])
    certified_edges.append([item[0]])
    for idx1 in item[1:]:
        if (ctr % 100 == 0):
            print(f"Doing {ctr}/{tot_num_edges}")
        s1 = PRM.nodes[idx1]
        t0 = time.time()
        safe = line_certifier.CertifyTangentConfigurationSpaceLine(s0,s1)
        t1 = time.time()
        if not safe:
            uncertified_edges[-1].append(idx1)
        else:
            certified_edges[-1].append(idx1)
            num_safe += 1
        ctr += 1
t1_tot = time.time()
print(f"Certification in {t1_tot-t0_tot}s")
print(f"Certified {num_safe}/{ctr} edges")

In [ ]:
plot_prm(PRM.nodes, certified_edges, 0.2, color = Rgba(0.0, 1.0, 0.0, 1), prefix = "const_certified_")
plot_prm(PRM.nodes, uncertified_edges, 0.2, color = Rgba(1.0, 0.0, 0.0, 1), prefix = "const_uncertified_")